In [1]:
import zipfile
import os

# Path to the uploaded ZIP file
zip_path = "cat_dog.zip"   # name of your uploaded file
extract_path = "cat_dog_extracted"  # folder to extract

# Extract
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check files
os.listdir(extract_path)


['train', 'train.csv', 'val', 'val.csv']

In [2]:
!pip install opencv-python

In [3]:
import os
import pandas as pd

# Load CSV
extract_path = "cat_dog_extracted"  # folder where ZIP is extracted
train_csv_path = os.path.join(extract_path, 'train.csv')
train_df = pd.read_csv(train_csv_path)

# Update image paths
train_df['image_path'] = train_df['image:FILE'].apply(lambda x: os.path.join(extract_path, x))

# Check first image path
print(train_df['image_path'][0])
print(os.path.exists(train_df['image_path'][0]))  # Should be True now


cat_dog_extracted\train/cat/Sphynx_159_jpg.rf.022528b23ac690c34ad5d109c1782079.jpg
True


In [4]:
!pip install Pillow

In [5]:
from PIL import Image
import numpy as np

In [6]:
img=Image.open(train_df['image_path'][0])
img=img.convert('L')

In [7]:
img=img.resize((64,64))

In [8]:
feature=np.array(img).flatten()

In [9]:
print("Feature vector shape:",feature.shape)
print("First 10 values:",feature[:10])

Feature vector shape: (4096,)
First 10 values: [13  7  4 41 87 95 58 25 28 34]


In [10]:
from tqdm import tqdm
X=[]
y=[]
for idx,row in tqdm(train_df.iterrows(),total=len(train_df)):
    img_path=row['image_path']
    label=row['category']
    img=Image.open(img_path).convert('L')
    img=img.resize((64,64))
    feature=np.array(img).flatten()
    X.append(feature)
    y.append(label)
X=np.array(X)
y=np.array(y)
print("Feature matrix shape:",X.shape)
print("Labels shape:",y.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 275/275 [00:05<00:00, 49.95it/s]

Feature matrix shape: (275, 4096)
Labels shape: (275,)


In [11]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [17]:
svm_model=SVC(kernel='linear',C=1.0)
svm_model.fit(X_train,y_train)

,C,1.0
,kernel,'linear'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [18]:
y_pred=svm_model.predict(X_test)

In [19]:
accuracy=accuracy_score(y_test,y_pred)
print("Accuracy:",accuracy)

Accuracy: 0.509090909090909


In [20]:
X_scaled=X/255.0

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


In [22]:
from sklearn.svm import SVC
svm_rbf=SVC(kernel='rbf',C=10,gamma='scale')

In [32]:
svm_rbf.fit(X_train, y_train)


,C,10
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


In [33]:
from sklearn.metrics import accuracy_score

y_pred_rbf = svm_rbf.predict(X_test)
accuracy_rbf = accuracy_score(y_test, y_pred_rbf)

print("RBF SVM Accuracy:", accuracy_rbf)


RBF SVM Accuracy: 0.6181818181818182
